In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import os
os.chdir('C://work/dev/python/progs/texts/sec_bert/')

In [3]:
import pandas as pd
import numpy as np
import joblib

from ruamel.yaml import YAML

conf = YAML().load(open('params.yaml'))
conf_conv = YAML().load(open('dvc_pipes/train_conv_emb/params_conv.yaml'))
conf_bert = YAML().load(open('dvc_pipes/train_bert_emb/params_bert.yaml'))

np.random.seed(conf['seed'])


In [16]:
import json

mitre_attack_df = pd.read_csv(conf['get_data']['data_mitre_attack_proc_fn'])
mitre_attack_df = mitre_attack_df.assign(labels = mitre_attack_df['labels'].map(lambda x:[x]))

# mitre_attack_df = mitre_attack_df[mitre_attack_df['sentence'].str.split().str.len()>=10].reset_index(drop=True)
# write same sentences for different technick 
mitre_attack_df[(mitre_attack_df.duplicated('sentence', keep=False))
        &(mitre_attack_df['sentence']!=' ')\
        &(mitre_attack_df['sentence'].str.split().str.len()>=10)\
        &(~mitre_attack_df['sentence'].str.contains('*', regex=False))].sort_values(by='sentence')\
        .to_csv('data/artifacts/duple_text_dif_techniks.csv')

# mitre_attack_df = mitre_attack_df.drop_duplicates('sentence')

with open(conf['get_data']['label2tactic_fn'], 'rt') as f_r:
    label2tactic = json.load(f_r)

tram_df = pd.read_json(conf['get_data']['tram_fn']).drop(columns='doc_title')
sel = tram_df.sentence.str.findall(';').str.len()>0
tram_df[sel].to_csv('data/artifacts/tram_rubbish.csv', index=False)
tram_df = tram_df[~sel]
tram_df = tram_df.drop_duplicates(subset='sentence')




if conf['get_data']['use_alt_mitre']:
    par_d = mitre_attack_df.explode('labels').set_index('labels')['par_name'].to_dict()
    mitr_alt_data = pd.read_csv(conf['get_data']['mitre_alt_fn'])
    mitr_alt_data['labels'] = mitr_alt_data[['label_subtec', 'label_tec']].apply(lambda x: list(set((x[0], x[1]))), axis=1)
    mitr_alt_data = mitr_alt_data[['sentence', 'labels']]
    mitr_alt_data['par_name'] = mitr_alt_data['labels'].map(lambda x: sorted([par_d[it] if it in par_d and len(it)>6 else '' for it in x])[-1])

    mitr_df = mitr_alt_data

else:
    mitr_df = mitre_attack_df

if conf['get_data']['use_tram_f']:
    df = pd.concat([mitr_df, tram_df], ignore_index=True)
else:
    df = mitr_df
    

In [ ]:
DN = conf['get_data']['rep_dn']

fns = [f'{DN}/{it}' for it in os.listdir(DN) if 'json' in it]


tab_df = pd.concat([pd.read_json(fn).explode('tables') for fn in fns], ignore_index=True)

tab_df['tables'] = tab_df['tables'].map(lambda x: [(it1, it2) 
                                for it1, it2 in zip(x['TechniqueID'].values(), x['Procedure'].values())])
tab_df = tab_df.explode('tables').drop_duplicates().reset_index(drop=True)
tab_df['technic'] = tab_df['tables'].map(lambda x: x[0])
tab_df['sentence'] = tab_df['tables'].map(lambda x: x[1])

tab_df = tab_df[tab_df['sentence'].str.split().str.len()>3]
tab_df['labels'] = tab_df['technic'].map(lambda x: [x.upper()])


label2tactic = joblib.load('data/label2tactic.pkl')

# тут и мобильные угрозы есть, поэтому фильтруем их
tab_df = tab_df[tab_df['labels'].map(lambda x: all([it in label2tactic for it in x]))]

In [20]:
pd.concat([df, tab_df[['sentence',	'labels', 'report_path']].rename(columns={'report_path':'url'})], ignore_index=True)

,sentence,labels,url,par_name,is_proc
0,Adversaries may inject malicious code into pro...,[T1055.011],https://attack.mitre.org/techniques/T1055/011,Process Injection,False
1,"Before creating a window, graphical Windows-ba...",[T1055.011],https://attack.mitre.org/techniques/T1055/011,Process Injection,False
2,"Although small, the EWM is large enough to sto...",[T1055.011],https://attack.mitre.org/techniques/T1055/011,Process Injection,False
3,Execution granted through EWM injection may al...,[T1055.011],https://attack.mitre.org/techniques/T1055/011,Process Injection,False
4,Running code in the context of another process...,[T1055.011],https://attack.mitre.org/techniques/T1055/011,Process Injection,False
...,...,...,...,...,...
35373,The backdoor uses an unencrypted binary protoc...,[T1095],/opt/nifi/configuration_resources/rstthreat/ti...,NaN,NaN
35374,Cyber Toufan s attacks often involve data extr...,[T1485],/opt/nifi/configuration_resources/rstthreat/ti...,NaN,NaN
35375,Cyber Toufan likely conducts active scanning t...,[T1595],/opt/nifi/configuration_resources/rstthreat/ti...,NaN,NaN
35376,"Given their operational scale, they may acquir...",[T1583],/opt/nifi/configuration_resources/rstthreat/ti...,NaN,NaN


In [9]:
import joblib
joblib.dump(tab_df, 'data/rep_data.pkl')

['data/rep_data.pkl']

# Аномалии

In [15]:
tab_df = tab_df[tab_df['sentence'].str.split().str.len()<3]

# Проблемы 

- есть дубли, случайно или что-то перезаписано (файл attck_procedures_202203, 0 элемента, 2 таблицы)
- короткие описания в пару слов, могут вводить в заблуждение модель (файл attck_procedures_202203, 1 элемент, 0 таблица)

# Старое

In [2]:
import pandas as pd
import numpy as np
np.random.seed(0)



In [19]:
import os
dn = 'data/reports_202212'
fns = [f'{dn}/{it}/model.json' for it in os.listdir(dn)]
# fn = fns[5]
fn

'data/reports_202212/20221202_decoded_avast_io_report_0x2ca02e2c/model.json'

In [18]:
import json

def get_report_paras(fn):
    rep_sents = [it for chapter in json.load(open(fn, 'rt'))['chapters'] for it in chapter['sentences']]
    rep_sents = [it['text'] for it in rep_sents]
    return rep_sents
    
rep_sents = get_report_paras(fn)

FileNotFoundError: [Errno 2] No such file or directory: 'data/reports_202212/20221202_decoded_avast_io_report_0x2ca02e2c/model.json'

In [20]:
import re
regexp_ttp = re.compile(r'(^| )(t\d{3,}(\.\d+)?)([., ]|$)', flags=re.IGNORECASE)
# regexp_ttp = r'[\d\.]{1,}'
for fn in fns:
    if os.path.exists(fn):
        rep_sents = get_report_paras(fn)
        text = '\n'.join(rep_sents)
        res_l = re.findall(regexp_ttp, text)
        if len(res_l)>0:
            print(fn)

data/reports_202212/20221202_asec_ahnlab_com_report_0x97bb2408/model.json
data/reports_202212/20221205_us-cert_cisa_gov_report_0xdc5d404f/model.json
data/reports_202212/20221223_trendmicro_com_report_0x203a8843/model.json


In [28]:
fn = 'data/reports_202212/20221223_trendmicro_com_report_0x203a8843/model.json'
rep_sents = get_report_paras(fn)
text = '\n'.join(rep_sents)
# print(text)
res_l = re.findall(regexp_ttp, text)
res_l

[(' ', 'T1218.007', '.007', ' '),
 (' ', 'T1218.011', '.011', ' '),
 (' ', 'T1218.011', '.011', ' '),
 (' ', 'T1027.009', '.009', ' '),
 (' ', 'T1218.011', '.011', ' ')]

In [ ]:
для всех sentences из каждой chapter



In [ ]:
поисковой механизм
- берем поабзацно и сравниваем